This is agent for marking the attendance to google sheets automatically after extracting roll numbers from marked sheet and remove manual works for teacher's assistant
Have Fun with it !

In [ ]:
!pip install opencv-python gspread oauth2client

In [3]:
import cv2
import pytesseract
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
import os
import json
import re
from google.colab.patches import cv2_imshow
from google import genai
from google.genai import types
import PIL.Image
from googleapiclient.discovery import build
from google.colab import userdata

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/MyDrive/images" # add your location of images in drive 

In [ ]:
import shutil

# Path to file in Google Drive
src_path = "/content/drive/MyDrive/credentials.json"

# Destination in Colab's local storage
dest_path = "/content/credentials.json" # add location of credentials.json

# Copy the file
shutil.copy(src_path, dest_path)

'/content/credentials.json'

In [7]:
json_path="/content/credentials.json"

In [ ]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open("test").sheet1  # Add your sheet name 

service = build("sheets", "v4", credentials=creds)

# Example: Read data
# data = sheet.get_all_records()
# print("Google Sheets Data:", data)

In [12]:
client = genai.Client(api_key=userdata.get('GEMINI_API'))
present_roll_numbers=[]
for img in os.listdir(folder_path):
    image = PIL.Image.open(os.path.join(folder_path, img))
    response=client.models.generate_content(
        model="gemini-2.0-flash",
        contents=['''Extract roll numbers from it they start with b or B and after then some numbers extract all the roll numbers and make a python list of them and return response as like this {"roll_numbers" : list of roll number},  here roll_numbers is a key and list of roll number is a list that you formed , Return the following data in pure JSON format (not wrapped in a code block). Do not include ```json or any markdown formatting. Only return a valid JSON dictionary. Example output:
    {"roll_numbers": ["B23423", "B23355", "B23054"]} , Output only valid JSON. No explanations, no markdown, no code block formatting. Just the raw JSON dictionary.?''',image]
    )
    json_string=response.text
    # Strip the code block markers
    json_string = json_string.strip("```json").strip("```").strip()
    data_dict = json.loads(json_string)
    present_roll_numbers.extend(data_dict["roll_numbers"])

In [17]:
present_roll_numbers = set(rn.upper() for rn in present_roll_numbers)

In [18]:
print(len(present_roll_numbers))

272


In [ ]:
data2=sheet.get_all_values()
headers=data2[0]
print(headers)
date=input("Enter the date i.e column name for attendance: ")
# date=" 04-2-25" # take manual input as date can have space according to name given to row
date_col = headers.index(date) + 1
roll_col_index = headers.index("Roll Number") + 1

# Mark their attendance

# Spreadsheet ID (extract from the sheet URL)
spreadsheet_id = userdata.get("sheet_id")  # Replace with your actual Spreadsheet ID

# Store cell updates for batch processing
requests = []
values = []

# Mark "P" for matching roll numbers and add formatting requests
for i in range(1, len(data2)):  # Skip header
    roll_number = data2[i][roll_col_index - 1].strip().upper()  # Get roll number from row
    if roll_number in present_roll_numbers:
        row_number = i + 1  # Convert to 1-based index

        values.append({"range": f"Sheet1!{chr(64 + date_col)}{row_number}", "values": [["P"]]})
        # Update "P" in the sheet
        # sheet.update_cell(row_number, date_col, "P")

        # Add formatting request to make cell green with black text
        requests.append({
            "repeatCell": {
                "range": {
                    "sheetId": sheet.id,
                    "startRowIndex": row_number - 1,
                    "endRowIndex": row_number,
                    "startColumnIndex": date_col - 1,
                    "endColumnIndex": date_col
                },
                "cell": {
                    "userEnteredFormat": {
                        "backgroundColor": {"red": 0, "green": 0.98, "blue": 0},  # Light green
                        "textFormat": {"foregroundColor": {"red": 0, "green": 0, "blue": 0}}  # Black text
                    }
                },
                "fields": "userEnteredFormat(backgroundColor,textFormat)"
            }
        })

# Batch update "P" values
if values:
    body = {"valueInputOption": "USER_ENTERED", "data": values}
    service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

# Apply formatting using batchUpdate
if requests:
    body = {"requests": requests}
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

print(f"Attendance marked successfully for {date} , total students were {len(present_roll_numbers)}!")

['Roll Number', 'Full Name', '', 'P', 'A', 'L', 'X', ' 21-1-25', ' 23-1-25', ' 28-1-25', ' 30-1-25', ' 04-2-25', '06/02/2025', '11/02/2025', '13/02/2025', '18/02/2025', '20/02/2025', '25/02/2025', '27/02/2025']
Attendance marked successfully for  04-2-25 , total students were 272!
